In [1]:
import lmfit
from lmfit import minimize, Parameters, fit_report
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import Analysis
from scipy.optimize import least_squares
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

In [3]:
with open('noslope.pkl', 'rb') as input:
    noslope = pickle.load(input)

In [5]:
def sigmoid(x,v0):
    return 1.0 / (1 + np.exp(np.array((-x+v0)/0.1,dtype='float')))


def residual(params, x, y, v0, eps_data):

    model_l = params[0] + params[1] * np.sin(x*params[2] + (params[3]+params[4]*np.pi)) + params[5] * (x - v0)
    model_r = params[0] + params[1] * np.sin(x*params[2] + params[3]) + params[5] * (x - v0)
    
    out_r = (y-model_r)*sigmoid(x,v0)
    out_l = (y-model_l)*(1.0-sigmoid(x,v0))
    
    return (out_r+out_l)/eps_data



In [6]:
def search(seq, t):  
    min = 0
    max = len(seq) - 1
    while True:        
        m = (min + max) // 2
        if max <= min:
            if np.abs(t-seq[m]) < np.abs(t-seq[m-1]):          
                return m
            else:
                return m-1        
        if seq[m] < t:
            min = m + 1
        elif seq[m] > t:
            max = m - 1
        else:
            return m

In [7]:
def regression(x_min, x_max, data, loss = 'linear', eps = 0.1, plot=False, x_scale = 1.0):
    
    i_min = search(data.x,x_min)
    i_max = search(data.x,x_max)+1
    
    if i_min < 0:
        i_min = 0
    if i_max > len(data.x):
        i_max = len(data.x)
    
    x = data.x[i_min:i_max]
    y = data.y[i_min:i_max]
    
    
    
    a = (np.percentile(y,99)-np.percentile(y,1))/2.0
    
    params = []
    bounds = ([],[])
    
    h = np.mean(y)
    params.append(h) #height
    bounds[0].append(h*0.99)
    bounds[1].append(h*1.01)
    
    params.append(a) #amplitude
    bounds[0].append(a*0.8)
    bounds[1].append(a*1.2)
    
    params.append(2*np.pi/10) #frequency
    bounds[0].append(0.6)
    bounds[1].append(0.77)
    
    params.append(0.0) #phase
    bounds[0].append(0.0)
    bounds[1].append(2*np.pi)
    
    params.append(np.random.uniform(0.1,1.9)) #delta phase
    bounds[0].append(0.0)
    bounds[1].append(2)
    
    params.append(0.0) #slope
    bounds[0].append(-0.02)
    bounds[1].append(0.02)
    
    v0 = (x_min+x_max)/2.0 
    
    
    out = least_squares(residual, x0 = params, bounds=bounds, loss=loss, x_scale = x_scale, args=(x , y, v0, eps))
    
    if plot:
        params = out['x']
        model_l = params[0] + params[1] * np.sin(x*params[2] + (params[3]+params[4]*np.pi)) + params[5] * (x - v0)
        model_r = params[0] + params[1] * np.sin(x*params[2] + params[3]) + params[5] * (x - v0)
        plt.plot(x,y)
        plt.plot(x,model_r,c='r')
        plt.plot(x,model_l,c='k')
    
    return out    

In [8]:
def regression_rand(x_min, x_max, data, loss = 'linear', eps = 0.1, plot=False, x_scale = 1.0):
        
    x = data.x
    y = data.y
    
    i_min = search(x,x_min)
    i_max = search(x,x_max)+1
    
    if i_min < 0:
        i_min = 0
    if i_max > len(x):
        i_max = len(x)
    
    x = x[i_min:i_max]
    y = y[i_min:i_max]
    
    
    
    a = (np.percentile(y,99)-np.percentile(y,1))/2.0
    
    params = []
    bounds = ([],[])
    
    h = np.mean(y)+np.random.randn()*0.1
    params.append(h) #height
    bounds[0].append(h*0.7)
    bounds[1].append(h*1.3)
    
    params.append(a+np.random.randn()*0.001) #amplitude
    bounds[0].append(a*0.8)
    bounds[1].append(a*1.2)
    
    params.append(2*np.pi/10+np.random.randn()*0.01) #frequency
    bounds[0].append(0.6)
    bounds[1].append(0.8)
    
    params.append(np.random.random()+0.001) #phase
    bounds[0].append(0.0)
    bounds[1].append(2*np.pi)
    
    params.append(np.random.uniform(0.1,1.9)) #delta phase
    bounds[0].append(0.0)
    bounds[1].append(2)
    
    params.append(np.random.randn()*0.001) #slope
    bounds[0].append(-0.001)
    bounds[1].append(0.001)
    
    v0 = (x_min+x_max)/2.0
    
    try:

        out = least_squares(residual, x0 = params, bounds=bounds, loss=loss, x_scale = x_scale, args=(x , y, v0, eps))

        if plot:
            params = out['x']
            model_l = params[0] + params[1] * np.sin(x*params[2] + (params[3]+params[4]*np.pi)) + params[5] * (x - v0)
            model_r = params[0] + params[1] * np.sin(x*params[2] + params[3]) + params[5] * (x - v0)
            plt.plot(x,y)
            plt.plot(x,model_r,c='r')
            plt.plot(x,model_l,c='k')
    
    except:
        out = regression_rand(x_min, x_max, data, loss = loss, eps = eps, x_scale = x_scale, plot = plot)
    
    return out    

In [9]:
def regression_nobounds(x_min, x_max, data, loss = 'linear', eps = 0.1, plot=False, x_scale = 1.0):
    
        
    x = data.x
    y = data.y
    
    i_min = search(x,x_min)
    i_max = search(x,x_max)+1
    
    if i_min < 0:
        i_min = 0
    if i_max > len(x):
        i_max = len(x)
    
    x = x[i_min:i_max]
    y = y[i_min:i_max]
    
    
    
    a = (np.percentile(y,99)-np.percentile(y,1))/2.0
    
    params = []
    bounds = ([],[])
    
    h = np.mean(y+np.random.randn()*0.1)
    params.append(h) #height
    bounds[0].append(-np.inf)
    bounds[1].append(np.inf)
    
    params.append(a+np.random.randn()*0.001) #amplitude
    bounds[0].append(-np.inf)
    bounds[1].append(np.inf)
    
    params.append(2*np.pi/10+np.random.randn()*0.01) #frequency
    bounds[0].append(-np.inf)
    bounds[1].append(np.inf)
    
    params.append(np.random.random()+0.001) #phase
    bounds[0].append(0.0)
    bounds[1].append(2*np.pi)
    
    params.append(np.random.uniform(0.1,1.9)) #delta phase
    bounds[0].append(0.0)
    bounds[1].append(2)
    
    params.append(np.random.randn()*0.001) #slope
    bounds[0].append(-np.inf)
    bounds[1].append(np.inf)
    
    v0 = (x_min+x_max)/2.0
    
    out = least_squares(residual, x0 = params, method = 'lm', loss=loss, x_scale = x_scale, args=(x , y, v0, eps))
    
    if plot:
        params = out['x']
        model_l = params[0] + params[1] * np.sin(x*params[2] + (params[3]+params[4]*np.pi)) + params[5] * (x - v0)
        model_r = params[0] + params[1] * np.sin(x*params[2] + params[3]) + params[5] * (x - v0)
        plt.plot(x,y)
        plt.plot(x,model_r,c='r')
        plt.plot(x,model_l,c='k')
    
    return out    

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
def regression_CV(x_min, x_max, data, loss = 'linear', eps = 0.1, plot=False, x_scale = 1.0):
    
    x = data.x
    y = data.y
    
    i_min = search(x,x_min)
    i_max = search(x,x_max)+1
    
    if i_min < 0:
        i_min = 0
    if i_max > len(x):
        i_max = len(x)
    
    x = x[i_min:i_max]
    y = y[i_min:i_max]
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
    
    
    a = (np.percentile(y_train,99)-np.percentile(y_train,1))/2.0
    
    params = []
    bounds = ([],[])
    
    h = np.mean(y_train)+np.random.randn()*0.1
    params.append(h) #height
    bounds[0].append(h*0.7)
    bounds[1].append(h*1.3)
    
    params.append(a+np.random.randn()*0.001) #amplitude
    bounds[0].append(a*0.8)
    bounds[1].append(a*1.2)
    
    params.append(2*np.pi/10+np.random.randn()*0.01) #frequency
    bounds[0].append(0.6)
    bounds[1].append(0.8)
    
    params.append(np.random.random()+0.001) #phase
    bounds[0].append(0.0)
    bounds[1].append(2*np.pi)
    
    params.append(np.random.uniform(0.1,1.9)) #delta phase
    bounds[0].append(0.0)
    bounds[1].append(2)
    
    params.append(np.random.randn()*0.001) #slope
    bounds[0].append(-0.001)
    bounds[1].append(0.001)
    
    v0 = (x_min+x_max)/2.0
    
    out = least_squares(residual, x0 = params, method = 'lm', loss=loss, x_scale = x_scale, args=(X_train , y_train, v0, eps))
    
    if plot:
        params = out['x']
        model_l = params[0] + params[1] * np.sin(x*params[2] + (params[3]+params[4]*np.pi)) + params[5] * (x - v0)
        model_r = params[0] + params[1] * np.sin(x*params[2] + params[3]) + params[5] * (x - v0)
        plt.plot(x,y)
        plt.plot(x,model_r,c='r')
        plt.plot(x,model_l,c='k')
    
    return out    

In [10]:
def multi_reg_param_ci(data, x_min, x_max, result, eps): 
    i_min = search(data.x,x_min)
    i_max = search(data.x,x_max)+1

    if i_min < 0:
        i_min = 0
    if i_max > len(data.x):
        i_max = len(data.x)

    N = float(i_max - i_min)
    k = len(result.x)

    x = data.x[i_min:i_max]
    y = data.y[i_min:i_max]

    v0 = (x_min+x_max)/2.0

    res = result.fun*eps

    sigma_r = 1/(N-k)*np.sum(res**2)
    
    inv_j = np.linalg.inv(np.dot(result.jac.transpose(),result.jac))

    cov = sigma_r*inv_j
    
    return np.sqrt(np.diag(cov))

In [11]:
def multi_regression(x_min, x_max, data, reg_func, n = 20, loss = 'linear', eps = 0.1, x_scale = 1.0, plot=False):
    ci_deltaphi = np.inf
    result_hold = None
    for _ in range(n):
        result = reg_func(x_min, x_max, data, loss = loss, eps = eps, x_scale=x_scale)
        if result.success:
            ci = multi_reg_param_ci(data,x_min,x_max,result,eps)
            if ci[4] < ci_deltaphi:
                ci_deltaphi = ci[4]
                result_hold = result
                
    if plot:
        plt.figure()
        
        i_min = search(data.x,x_min)
        i_max = search(data.x,x_max)+1
    
        x = data.x[i_min:i_max]
        y = data.y[i_min:i_max]
        
        v0 = (x_min+x_max)/2.0
        
        params = result_hold.x
        model_l = params[0] + params[1] * np.sin(x*params[2] + (params[3]+params[4]*np.pi)) + params[5] * (x - v0)
        model_r = params[0] + params[1] * np.sin(x*params[2] + params[3]) + params[5] * (x - v0)
        plt.plot(x,y)
        plt.plot(x,model_r,c='r')
        plt.plot(x,model_l,c='k')
    return result_hold, ci_deltaphi

In [12]:
def sweep_regression(data, reg_func, d = 6.0, c = 20, n = 20, loss = 'linear', eps = 0.1, x_scale = 1.0, plot=False):
    out = list()
    x_max = max(data.x) - 0.005
    x_min = min(data.x) + 0.005
    #step = (x_max-x_min-2*d)/c
    step = (x_max-x_min)/c
    for i in range(c+1):
        #Try to start at edge with half
        #center = x_min + d +  i*step
        center = x_min +  i*step
        low = center - d
        if low < x_min:
            low = x_min
        high = center + d
        if high > x_max:
            high = x_max
        result, ci_deltaphi = multi_regression(low, high, data, reg_func, n, loss = loss, eps = eps, x_scale = x_scale, plot = plot)
        
        res_dict = {'result':result, 'x_min':low, 'x_max':high, 'eps':eps, 'ci_deltaphi':ci_deltaphi}
        
        out.append(res_dict)
    return out
    

In [13]:
def run_regression(data, reg_func, d = 6.0, c = 20, n = 20, loss = 'linear', eps = 0.01, x_scale = 1.0, plot=False):
    count = 0
    out = []
    for i in data:
        print('Running sweep ' + str(count))
        out.append(sweep_regression(i, reg_func, d = d, c = c, n=n, eps = eps, x_scale = x_scale, loss = loss, plot = plot))
        count += 1
    return out

In [14]:
def param_ci(data,result):
    x_min = result['x_min']
    x_max = result['x_max']
    i_min = search(data.x,x_min)
    i_max = search(data.x,x_max)+1

    if i_min < 0:
        i_min = 0
    if i_max > len(data.x):
        i_max = len(data.x)

    N = float(i_max - i_min)
    k = len(result['result'].x)

    x = data.x[i_min:i_max]
    y = data.y[i_min:i_max]

    v0 = (x_min+x_max)/2.0

    res = result['result'].fun*result['eps']

    sigma_r = 1/(N-k)*np.sum(res**2)
    
    inv_j = np.linalg.inv(np.dot(result['result'].jac.transpose(),result['result'].jac))

    cov = sigma_r*inv_j
    
    return np.sqrt(np.diag(cov))
    


In [15]:
def plot(data,result,full=False):
    x_min = result['x_min']
    x_max = result['x_max']
    if full:
        x = data.x
        y = data.y
    else:
        i_min = search(data.x,x_min)
        i_max = search(data.x,x_max)+1

        x = data.x[i_min:i_max]
        y = data.y[i_min:i_max]

    v0 = (x_min+x_max)/2.0

    params = result['result'].x
    model_l = params[0] + params[1] * np.sin(x*params[2] + (params[3]+params[4]*np.pi)) + params[5] * (x - v0)
    model_r = params[0] + params[1] * np.sin(x*params[2] + params[3]) + params[5] * (x - v0)

    plt.plot(x,y)
    plt.plot(x,model_r,c='r')
    plt.plot(x,model_l,c='k')

In [16]:
def r_squared(data,result):
    x_min = result['x_min']
    x_max = result['x_max']
    i_min = search(data.x,x_min)
    i_max = search(data.x,x_max)+1
    
    if i_min < 0:
        i_min = 0
    if i_max > len(data.x):
        i_max = len(data.x)

    x = data.x[i_min:i_max]
    y = data.y[i_min:i_max]

    v0 = (x_min+x_max)/2.0
    
    res = result['result'].fun*result['eps']
    SSres = np.sum(res*res)
    
    ybar = np.mean(y)
    
    SStot = np.sum((y-ybar)**2)

    return 1 - SSres/SStot

In [22]:
run = Analysis.generate_2014()

In [24]:
test = regression(-19.5464, -8.39718,run.prune_data[0],eps = 1.0, plot = True)

In [64]:
test_multi = multi_regression(-19.5464, -8.39718, run.prune_data[0], eps = 0.01, plot = True, x_scale = [10.0,0.1,0.1,1.0,1.0,1.0], loss = 'huber')

In [117]:
test_sweep_reg = sweep_regression(run.prune_data[0], d = 4.5, c = 50, eps = 0.01, x_scale = [10.0,0.1,0.1,1.0,1.0,1.0], loss = 'huber')

In [157]:
run_reg = run_regression(run.prune_data, regression, d = 4.5, c = 50, eps = 0.01, x_scale = [10.0,0.1,0.1,1.0,1.0,10**-5], loss = 'huber')

Running sweep 0
Running sweep 1
Running sweep 2
Running sweep 3
Running sweep 4
Running sweep 5
Running sweep 6
Running sweep 7
Running sweep 8
Running sweep 9
Running sweep 10
Running sweep 11
Running sweep 12
Running sweep 13
Running sweep 14


In [27]:
run_reg_rand = run_regression(run.prune_data, regression_rand, d = 5.5, c = 50, eps = 0.01, x_scale = [10.0,0.1,0.1,1.0,1.0,10**-3], loss = 'huber')

Running sweep 0
Running sweep 1
Running sweep 2
Running sweep 3
Running sweep 4
Running sweep 5
Running sweep 6
Running sweep 7
Running sweep 8
Running sweep 9
Running sweep 10
Running sweep 11
Running sweep 12
Running sweep 13
Running sweep 14


In [22]:
run_reg_rand_smallerwindow = run_regression(run.prune_data, regression_rand, d = 4.0, c = 50, eps = 0.01, x_scale = [10.0,0.1,0.1,1.0,1.0,10**-3], loss = 'huber')

Running sweep 0
Running sweep 1
Running sweep 2
Running sweep 3
Running sweep 4
Running sweep 5
Running sweep 6
Running sweep 7
Running sweep 8
Running sweep 9
Running sweep 10
Running sweep 11
Running sweep 12
Running sweep 13
Running sweep 14


In [37]:
np.save('2014_scipy_run_rand_tightslope_includeedge_smallwindow',run_reg_rand_smallerwindow)

In [17]:
run_reg_rand = np.load('2014_scipy_run_rand_tightslope_includeedge_smallwindow.npy')

In [18]:
filenum = 3

In [20]:
t = run_reg_rand[filenum]

In [19]:
filenum = 0
t = run_reg_rand[filenum]
hold = list()
for i in t:
    hold.append(i['ci_deltaphi'])
    
plt.plot(hold)

In [23]:
filenum = 3
run.prune_data[filenum].plot()

In [20]:
filenum = 3
t = run_reg_rand[filenum]
count = 0
for i in t:
    if i['ci_deltaphi'] < 0.001 and 0.1 < i['result'].x[4] < 1.9:
        plt.figure()
        plot(run.prune_data[filenum],i)
        plt.title(str(count) + ', '+ str(i['ci_deltaphi']) + ', '+ str(i['result'].x[4]))
    count += 1

In [31]:
plt.figure()
run.prune_data[filenum].plot()

In [168]:
plt.figure()
t = test_run_reg[filenum]
plot(run.prune_data[filenum],t[32])

In [23]:
for i in range(49,51):
    plt.figure()
    plot(run.prune_data[filenum],t[i])

In [27]:
plot(run.prune_data[filenum],t[i],full=True)

In [36]:
t[50]

{'ci_deltaphi': 0.0022057385940080555,
 'eps': 0.01,
 'result':  active_mask: array([ 0, -1, -1,  0,  0,  1])
        cost: 3436.0282782675813
         fun: array([ 2.35925935,  2.33473977,  0.31110951, ..., -3.80727092,
       -1.25846849, -0.35061096])
        grad: array([  7.06826444e-06,   1.60111941e+03,   3.12624277e+02,
        -9.97835598e-07,  -3.71549630e-04,  -2.68466638e+02])
         jac: array([[ -1.49011612e-06,  -1.46999212e-06,   3.41831448e-07,
         -4.28311179e-08,  -1.34557984e-07,   2.98164640e-06],
       [ -1.49011612e-06,  -1.47017935e-06,   3.40194357e-07,
         -4.26328380e-08,  -1.33935034e-07,   2.97973752e-06],
       [ -1.00000000e+02,  -9.86755725e+01,   2.27113721e+01,
         -2.84666599e+00,  -8.94306896e+00,   1.99828398e+02],
       ..., 
       [ -1.49011612e-06,   2.18761367e-07,   1.02985140e-06,
         -2.58665751e-07,   1.41162557e-15,  -2.97807894e-06],
       [ -1.49011612e-06,   2.17848496e-07,   1.02967874e-06,
         -2.5868944

In [49]:
plt.plot(t[19]['result'].fun)

In [158]:
test_run_reg_noslope = run_regression(noslope.prune_data, regression_nobounds, d = 4.5, c = 50, eps = 0.01, x_scale = [10.0,0.1,0.1,1.0,1.0,10**-5], loss = 'linear')

Running sweep 0
Running sweep 1
Running sweep 2
Running sweep 3
Running sweep 4
Running sweep 5
Running sweep 6
Running sweep 7
Running sweep 8
Running sweep 9
Running sweep 10
Running sweep 11
Running sweep 12
Running sweep 13
Running sweep 14


In [159]:
test_run_reg_rand = run_regression(noslope.prune_data, regression_rand, d = 4.5, c = 50, eps = 0.01, x_scale = [10.0,0.1,0.1,1.0,1.0,10**-5], loss = 'huber')

Running sweep 0
Running sweep 1
Running sweep 2
Running sweep 3
Running sweep 4
Running sweep 5
Running sweep 6
Running sweep 7
Running sweep 8
Running sweep 9
Running sweep 10
Running sweep 11
Running sweep 12
Running sweep 13
Running sweep 14


In [16]:
run_reg = np.load('scipy_run.npy')
run_rand = np.load('scipy_run_rand.npy')

In [107]:
plot(run.prune_data[filenum],test_run_reg_noslope[filenum][36],True)

In [174]:
filenum = 2
t = run_reg_noslope[filenum]
count = 0
for i in t:
    if i['ci_deltaphi'] < 0.001 and 0.1 < i['result'].x[4] < 1.9:
        plt.figure()
        plot(run.prune_data[filenum],i)
        plt.title(str(count) + ', '+ str(i['ci_deltaphi']) + ', '+ str(i['result'].x[4]))
    count += 1

In [124]:
plt.figure()
plot(noslope.prune_data[filenum],t[12],True)

In [175]:
filenum = 2
t = run_reg[filenum]
count = 0
for i in t:
    if i['ci_deltaphi'] < 0.001 and 0.1 < i['result'].x[4] < 1.9:
        plt.figure()
        plot(run.prune_data[filenum],i)
        plt.title(str(count) + ', '+ str(i['ci_deltaphi']) + ', '+ str(i['result'].x[4]))
    count += 1

In [55]:
plt.figure()
t = run_reg[filenum]
plot(run.prune_data[filenum],t[37],full=True)

In [64]:
filenum = 0
t = run_rand[filenum]
count = 0
for i in t:
    if i['ci_deltaphi'] < 0.001 and 0.1 < i['result'].x[4] < 1.9:
        plt.figure()
        plot(noslope.prune_data[filenum],i)
        plt.title(str(count) + ', '+ str(i['ci_deltaphi']) + ', '+ str(i['result'].x[4]))
    count += 1